<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Analyzing Proximity to Climate Risk Data Loading</b>
</header>

<br>

<p style = 'font-size:16px;font-family:Arial'>Notebook to load Geospatial data into destination Vantage System.  There are a few options:</p>

<p style = 'font-size:16px;font-family:Arial'><b>Teradata GeoImport Utility</b> <br> Requires Windows and Java; supports multiple import formats including TIGER, ESRI, MAPINFO, ESRI File Geodatabase.  For this demo, use the following import commands:</p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>java -Xms256m -Xmx512m com.teradata.geo.TDGeoImport -logmech LDAP -l tdprd.td.teradata.com/aj255038,password -s AJ255038 -f "C:\Users\AJ255038\Downloads\ANZ Geospatial\esri_data\nz_floods" -n nz_floods</li>
    <li>java -Xms256m -Xmx512m com.teradata.geo.TDGeoImport -logmech LDAP -l tdprd.td.teradata.com/aj255038,password -s AJ255038 -f "C:\Users\AJ255038\Downloads\ANZ Geospatial\esri_data\nz_addrs" -n nz_street_address</li>
    </ul>
<p style = 'font-size:16px;font-family:Arial'>See the <a href = 'https://docs.teradata.com/r/Teradata-Geospatial-Utilities-User-Guide/June-2022/Teradata-Geospatial-Utilities-Overview'>documentation</a> for more details</p>

<p style = 'font-size:16px;font-family:Arial'><b>GeoPandas and teradataml</b>  <br>For smaller volumes of data, Python, GeoPandas, and teradataml methods can be used to load data.  This is the process outlined in this notebook:</p>

<ol style = 'font-size:16px;font-family:Arial'>
    <li>Python Package Imports.  Requires teradataml, geopandas, and shapely packages.</li>
    <li>CSV Files have been included in this demo. If you are creating new data sets from ESRI Shapefiles, or other geospatial formats, one can use GeoPandas to read the files, and export a csv file with Well Known Text Format for the geometry objects.</li>
    <li>Load NZ Address Data:<ol style = 'font-size:16px;font-family:Arial'>
        <li>Connect using teradataml</li>
        <li>Create an empty table</li>
        <li>Read csv file using Pandas to fix up NaN column issues with teradataml</li>
        <li>Use the FastLoad method to load data</li>
        <li>Load the Flood Zones csv file directly.  No need to massage the data</li>
        <li>Disconnect</li></ol></li>
    <li>Create Views to convert the WKT column to ST_Geometry type:<ol style = 'font-size:16px;font-family:Arial'>
        <li>Connect to Vantage</li>
        <li>Execute DDL</li>
        <li>Disconnect</li>
        </ol></li>
    </ol>
        

In [1]:
import teradatasql, json
from teradataml import *



In [7]:
# load vars json
with open('../../vars.json', 'r') as f:
    session_vars = json.load(f)

# Create the SQLAlchemy Context
host = session_vars['environment']['host']
name = session_vars['hierarchy']['SYSDBA']['username']
pwd = session_vars['hierarchy']['SYSDBA']['password']

bucket = session_vars['environment']['bucket']
access_key = session_vars['environment']['access_key']
secret = session_vars['environment']['secret']

# eng = create_context(host=host, username=username, password=password)
# print(eng)

In [10]:

# Create nz_addrs Table

with teradatasql.connect(host = session_vars['environment']['host'], 
                     user = name, 
                     password = pwd) as con:
    cur = con.cursor()

    try:
        cur.execute('DROP TABLE demo.nz_addrs')
    except Exception as e:
        # Table doesn't exist
        if str(e.args).find("3807") >= 1:
            pass
        else:
            raise
        
    qry = '''
    CREATE MULTISET TABLE demo.nz_addrs ,NO FALLBACK ,
         NO BEFORE JOURNAL,
         NO AFTER JOURNAL,
         CHECKSUM = DEFAULT,
         DEFAULT MERGEBLOCKRATIO,
         MAP = TD_MAP2
         (
          address_id BIGINT,
          change_id BIGINT,
          address_ty VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          unit_value VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          address_nu BIGINT,
          address__1 VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          address__2 FLOAT,
          water_rout VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          water_name VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          suburb_loc VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          town_city VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          full_addre VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          full_road_ VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          full_add_1 VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          road_secti BIGINT,
          gd2000_xco FLOAT,
          gd2000_yco FLOAT,
          water_ro_1 VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          water_na_1 VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          suburb_l_1 VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          town_city_ VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          full_roa_1 VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          full_add_2 VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
          geometry VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC)
    NO PRIMARY INDEX ;'''

    cur.execute(qry)


    qry = f'''
    Insert into demo.nz_addrs 
    SELECT address_id, change_id, address_ty, unit_value, address_nu,
            address__1, address__2, water_rout, water_name, suburb_loc, town_city, 
            full_addre, full_road_, full_add_1, road_secti, gd2000_xco, gd2000_yco, 
            water_ro_1, water_na_1, suburb_l_1, town_city_, full_roa_1, full_add_2, geometry
    FROM
            (
                LOCATION = '/s3/{bucket}.s3.amazonaws.com/DEMO/NZ_ADDRS/manifest.json'
                AUTHORIZATION = '{{"Access_ID":"{access_key}","Access_Key":"{secret}"}}'
                MANIFEST = 'TRUE'
            ) D;'''
    cur.execute(qry)

In [11]:

# Create nz_addrs Table

with teradatasql.connect(host = session_vars['environment']['host'], 
                     user = name, 
                     password = pwd) as con:
    cur = con.cursor()

    try:
        cur.execute('DROP TABLE demo.nz_floods')
    except Exception as e:
        # Table doesn't exist
        if str(e.args).find("3807") >= 1:
            pass
        else:
            raise
        
    qry = '''
    CREATE MULTISET TABLE demo.nz_floods ,FALLBACK ,
         NO BEFORE JOURNAL,
         NO AFTER JOURNAL,
         CHECKSUM = DEFAULT,
         DEFAULT MERGEBLOCKRATIO,
         MAP = TD_MAP2
         (
          fid FLOAT,
          id FLOAT,
          geometry VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC)
    NO PRIMARY INDEX ;'''

    cur.execute(qry)


    qry = f'''
    Insert into demo.nz_floods 
    SELECT fid, id, geometry
    FROM
            (
                LOCATION = '/s3/{bucket}.s3.amazonaws.com/DEMO/NZ_FLOODS/manifest.json'
                AUTHORIZATION = '{{"Access_ID":"{access_key}","Access_Key":"{secret}"}}'
                MANIFEST = 'TRUE'
            ) D;'''
    cur.execute(qry)

## Create ST_Geometry Views ##
Use a separate connection, since fastload() seems to create issues with transactions

In [12]:


# load vars json
with open('../../vars.json', 'r') as f:
    session_vars = json.load(f)

name = session_vars['hierarchy']['SYSDBA']['username']
pwd = session_vars['hierarchy']['SYSDBA']['password']

with teradatasql.connect(host = session_vars['environment']['host'], 
                     user = name, 
                     password = pwd) as con:
    cur = con.cursor()

    qry = '''
    REPLACE VIEW demo.nz_addrs_geoV
         AS
         SELECT
          address_id, change_id, address_ty, unit_value, address_nu, address__1,
          address__2, water_rout, water_name, suburb_loc, town_city, full_addre,
          full_road_, full_add_1, road_secti, gd2000_xco, gd2000_yco, water_ro_1,
          water_na_1, suburb_l_1, town_city_, full_roa_1,full_add_2,
          NEW ST_GEOMETRY("geometry") AS POINT
        FROM demo.nz_addrs;'''
    cur.execute(qry)
    
    qry = '''
    REPLACE VIEW demo.nz_floods_geoV
         AS
         SELECT
          fid, id,
          NEW ST_GEOMETRY("geometry") AS GEOM
        FROM demo.nz_floods;'''
    cur.execute(qry)  